### Notebook to use 2 or more B&K DMMs at the same time and collect data from all B&Ks
### Mod Lab application:  Photoelectric Effect lab, measure stopping voltage and photocurrent
###  
#### Recommendation:  Turn on PE box and voltmeters for at least 20 minutes before taking data.  Typically it takes 30 minutes for instruments and power supplies to stabilize thermally.  When taking data, you'll get better results scanning from high stopping voltage toward low, because the photocurrents are smaller, causing less heating of the electronics.

#### You need to direct the light from the laser pointer straight into the opening in the electronics (PE Effect) box.  The best way to see that you are on target is to dial the stopping voltage down to zero and view the photocurrent signal.  To either side it will be very large.  Dead in the middle you will get a very small signal, because there is a mask inside that prevents light from falling directly on the anode (which would give you a reverse PE effect that we don't want to look at today), and that mask will also block the light from falling on the photocathode.  Just direct the light to either side, where you get a large photocurrent.  Zero the meter with the Zero knob, then turn the voltage all the way up and you should see the signal go away.  
#### When you actually take data you'll want to dial the knob slowly in some places and quickly in others, because we care mostly about the transition between essentially no photocurrent and just a tiny bit of photocurrent.  Because of the slow data rate of the DMMs, you'll want to make sure to give the meters enough time to settle at each voltage.

#### Work in semi-darkness to minimize stray light. Room lights, or light from your computer will give some contaminating wavelengths.   A cardboard tube should block room lights effectively.  It might be a nice idea to run a scan with the laser turned off in order to see that your baseline is at most a gently sloped straight line (due to differential heating in the circuitry causing baseline drift).  Then when you direct the diffused laser light, you'll see that the baseline shifts a bit, because the laser light heats the photcathode ever so gently.  Allow a couple of minutes for this baseline to settle.  Optionally, you can zero out the baseline again.

### Suggestion: reduce the size of this notebook window, and place it in the upper right hand corner of your screen.  The live plot of your data will appear in the upper left hand corner, rather than get lost beneath the Jupyter notebook icon in the taskbar.

In [2]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


### Code to establish communications between the PC and the B&K DMMs
#### For simplicity, you'll need to run this each time you take data in the cell that follows, in order to reestablish communications.  (You can try commenting out the port closing lines at the bottom of the acquisition routine, but if you quit the scan early the B&Ks get out of phase in reading.  If that happens, just run the cell to reestablish communications.)

In [63]:
# PJHT 9/22/17
import serial  #pySerial folder needs to be put in the directory pythonXX\lib\site-packages\serial
#The standard installation lacks this serial package.  If this notebook reports back that serial package
#is not found, open an Anaconda command window, then type
#>>>pip install pyserial
#then check that it's there with 
#>>>pip list
#The Anaconda 3 installer should put it in the correct directory

#Cabling: connect the B&Ks via usb cables.  Drivers may need to be installed from the B&K website
#Because the device registers as a virtual comm port, Windows will put it at some port #

import numpy as np
from numpy import append #need this to gather the data on the fly
from time import sleep, localtime,strftime #some timekeeping functions
import time
from serial import SerialException
from serial.tools import list_ports #to give you an iterator to list the ports
BAUD_BK = 38400  #The default rate is 9600 baud
'''
    This is the approximate transmission rate in bits per second.  Typically it takes about 10 bits to transmit
    a single character (1 start bit + 8 bits for the character + 1 or 2 stop bits), so figure that 
    9600 baud will do about 960 characters per second.

'''

def send_to_BK(command_str):  #This function a little different from the last (thermistor) lab
    # Now command_str is a list, into which we can put commands to send to all B&Ks in the same function
    #print("Inside send_to_BK, num = ", num)
    #print("Command String: ",command_str)
    return_bytes = [] #make it an empty list
    for BK in range(len(command_str)):
        #BK is simply an integer that will be the index of the corresponding B&K device
        # If BK = 0, it'll be the first device; BK = 1 is the second one, etc.
        command_str[BK] = command_str[BK]+"\n"  #append a <LF>, required for B&K
        err = ser_BK[BK].write(command_str[BK].encode())
    #   the .encode() method converts the string into a byte array, and is required to use ser.write() routine
    sleep(SLEEP_TIME) #typically this is 0.2 seconds, because the B&K is a bit slow
    for BK in range(len(command_str)):  #Now read back all connected B&Ks and store in big byte array
        #The .extend method is analogous to the string .append method, but is used to with byte arrays
        return_bytes.append(ser_BK[BK].read(DATA_LENGTH))
    sleep(SLEEP_TIME)
    #Uncomment this next line if you want to see that return_bytes now contains responses from all the 
    #connected B&K DMMs
    #print("All of return bytes: <<<<", return_bytes,">>>>\n\n")
    return return_bytes  #For two connected devices, this should return a pair of byte arrays

# We need some way to parse (chop up) the return_bytes byte arrays into responses from each device
# Fortunately we know that the identification information is four elements separated by commas:
def parse_ID(return_bytes): #
    #print("top of parse_ID",return_bytes)
    response_list = []
    return_string =[]
    
    for BK in range(NumBKports):
        #print("BK = ",BK)
        return_string.append(str(return_bytes[BK],'utf-8'))  #convert the bytes into a string
                                                    #convert the bytes into a string
    #print("Return string1 :",return_string)
    for BK in range(NumBKports):
        response_list = return_string[BK].split(",")   #so you can split up its parts on commas
        #print("Response_list split on comma",response_list)
        print("\n\rIdentification of device {:d}:\n\r".format(BK))
        print("\t",response_list[0])
        print("\t",response_list[1])
        print("\t",response_list[2])
       
    return 



TIME_OUT = 0.5  # don't hang around forever looking for data, but give the B& K time to respond

#This timeout limits the data rate
SLEEP_TIME = 0.3 #a pause for each time you write to or read from the B&K
DATA_LENGTH = 999  #could put this at 9999 and leave it

filename = "Data Logger Lab 5.txt"
headertext = 'Take data at regular intervals'
open(filename, mode ='w')

#Take a look at the connected ports:
ports =serial.tools.list_ports.comports()
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("Table of all connected COM ports:\n\r")
#Sort the list and print it
for p in sorted(ports):
    print(p)
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\r")

#ports has various attributes which can be accessed by index as shown in some examples here,
#which may look a little strange because the ports are not sorted in this view:
#print("\nHere's perhaps the first port: ",ports[0])
#print("And if there is a second port: ",ports[1])
#print("The port device name for ports[2]: ",ports[2].device)
#print("The description of the device on ports[2]: ",ports[2].description)
#print("Technical descriptor of ports[2]: \n", ports[2].hwid)
#print("  OK, that was full geek\n\n")
#More systematically 
#Find the ports that use the Silicon Labs CP210X controller inside the B&K usb port
#The B&K meters have a Silicon Labs interface, so let's look for this:
SilLabs = "Silicon Labs CP210x USB to UART Bridge" 
BKports =[] #an empty list that will be filled with full port descriptors
for p in sorted(ports):
    if p.description.find(SilLabs) >= 0:  #will report -1 if not there
        print("Found a Silicon Labs interface on port: ", p.device)
        BKports.append(p.device)

NumBKports = len(BKports) #the number of B&K ports
print("\nYou have {:d} connected BK ports, \n".format(NumBKports), BKports)

connected = []
ser_BK = [] #this will be the devices that you can communicate with

#return_bytes=bytearray() 


try:  #strategy:  for each port, build up a list of commands.  In this case we loop only
    #over valid B&K ports (NumBKports) and set their communication parameters
    for BK in range(NumBKports): 
        test = serial.Serial(BKports[BK],BAUD_BK,timeout=TIME_OUT, inter_byte_timeout = 0.005) 
        #print(test)
        ser_BK.append(test)
    
except SerialException:   
    print ("Port already open....closing...opening")
    ser_BK[BK].close()
sleep(SLEEP_TIME)
#Now that the ports are open, we see who's there
BK_str =[] #this will hold the strings of commands that you will send to the B&Ks
for BK in range(NumBKports):  #We build up a list of commands to send to all devices in turn
    #BK is simply an integer that will be the index of the corresponding B&K device
    # If BK = 0, it'll be the first device; BK = 1 is the second one, etc.
    #print("BK = ",BK)
    BK_str.append("*IDN?")  #This is the command to identify the device
    #So by the end of this loop, for 2 B&Ks, we'll have created a list of commands
#print("Check now that BK_str =", BK_str)
return_bytes = send_to_BK(BK_str) #after sending off the list of commands (1 per device)
                                #read back a list of responses for all BKs
#print("<<<", return_bytes,">>>\n\n\n")    
#print("Let's see what the responses are:\n")
try:
    parse_ID(return_bytes)
except:
    print("Error:  Make sure that both BK DMMs are turned on")
    
#print("return bytes outside: ",return_bytes)
#return_bytes = send_to_BK("*IDN?",i)      #who are you?
sleep(SLEEP_TIME)
#print(return_bytes)
#return_string = str(return_bytes,'utf-8')  #convert the bytes into a string
#response_list = return_string.split(",")   #so you can split up its parts on commas

#print("Connecting to B&K DMM on "+BKports[i])


#Flush the ports, again building up the new command list
print("Flushing all BK ports...")
for BK in range(NumBKports):
    ser_BK[BK].flushInput() #flush input buffer, discarding all its contents
sleep(SLEEP_TIME)
for BK in range(NumBKports): 
    ser_BK[BK].flushOutput()#flush output buffer, aborting current output 
sleep(SLEEP_TIME)
print("Resetting all BK devices...")
#Build up the list of commands
command_str=[] #clear out the previous command_str!
for BK in range(NumBKports):
    command_str.append("*rst")
return_bytes=send_to_BK(command_str)  #reset all devices
sleep(4) #a good 4 second delay to allow each device to reset itself
#Now set the functions
command_str=[] #clear out the previous command_str!
for BK in range(NumBKports):  #let everyone be a voltmeter
    command_str.append("func volt:dc")  
return_bytes=send_to_BK(command_str)  #
command_str=[] #clear out the previous command_str!
for BK in range(NumBKports):  #let everyone have the same response speed
    command_str.append("volt:dc:nplc 1")  #make it average 1 power line cycles, i.e. 1/60 s
return_bytes=send_to_BK(command_str)  #
command_str=[] #clear out the previous command_str!
for BK in range(NumBKports):  #let everyone be autoranging
    command_str.append("volt:dc:rang:auto on")  
return_bytes=send_to_BK(command_str)  #
print("\n\rBK DMMs are initialized\n\r")

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Table of all connected COM ports:

COM1 - Communications Port (COM1)
COM3 - Intel(R) Active Management Technology - SOL (COM3)
COM4 - Silicon Labs CP210x USB to UART Bridge (COM4)
COM7 - Silicon Labs CP210x USB to UART Bridge (COM7)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Found a Silicon Labs interface on port:  COM4
Found a Silicon Labs interface on port:  COM7

You have 2 connected BK ports, 
 ['COM4', 'COM7']

Identification of device 0:

	 *IDN?
5491B  Multimeter
	 Ver1.4.14.06.18
	 124E16106


Identification of device 1:

	 *IDN?
5491B  Multimeter
	 Ver1.4.14.06.18
	 124D17188

Flushing all BK ports...
Resetting all BK devices...

BK DMMs are initialized



### Main data acquisition loop
#### Be sure to change the wavelength appropriately so that your graphs contain that info.  The line plt.clf() erases the multicolored raw data plot each time.  If you want to put all your raw data on one plot, you could comment this line out.  It's handy for seeing the variation between two or more data sets.

In [64]:
#The plt.ion() command allows "interactive plotting on"  Look in the Taskbar for the plot 
# Scatter(v1, v2) plots the current list of voltages
#  You need a little pause in the plotting to get this to show
#  (This method of plotting while you take data is frowned upon and will generate a warning at the bottom of this cell.  Just
#  ignore it until Python comes up with something better!)

# You'll note that this cell will take 100 data points before stopping.

wavelength = 446 #the wavelength (nm) of the light source
f_out_root = "Blue2"  #pick a filename root
time_string = strftime("%Y.%m.%d.%H.%M.%S",localtime())
f_out_root = f_out_root+time_string+".txt" #and then make it unique by writing the time with it
f_out = open(f_out_root, mode ='w') #and open a file handle for writing
v1_list=[]  #keep track of two devices
v2_list=[] # 
time_list=[] #and the time, which will store in numerical format; convert later
done_flag = 0
count = 0
MAX_COUNT=60*60*24  #good for 24 hours if 1 sec per loop
MAX_COUNT = 100  #just try it for 100 values


fig1 = plt.figure(1, figsize = (6,6))  #select a 6 inch square figure
f_manager = plt.get_current_fig_manager()
f_manager.window.move(0,0)  #place figure in upper left corner
##################################################################
plt.clf() #clear out the old data
##################################################################
title("Raw data at {:4.1f} nm".format(wavelength))
xlabel("Stopping Potential")
ylabel("Voltage proportional to photocurrent")
plt.ion()
print("Scanning...",end ="\r")
bad_count = 0
for BK in range(NumBKports):       
        BK_str[BK] = "FETC?"  #This is the command to fetch the next fresh data

while (count< MAX_COUNT): 
    return_string=[]
    trunc_string=[]
    #sleep(1) #this pause, plus the pauses within send_to_BK, determine the acquisition period
    
    return_bytes = send_to_BK(BK_str) #this now returns a list of bytearrays
    for BK in range(NumBKports):
        #print("BK = ",BK)
        return_string.append(str(return_bytes[BK],'utf-8'))  #convert the bytes into a string
                                                    #convert the bytes into a string
    #print("return_bytes: ",return_bytes)
    #return_string = str(return_bytes,'utf-8')  #convert the bytes into a string (handy for further manipulation)
    #print("return_string: ",return_string)
    try:
        for BK in range(NumBKports):
            trunc_string.append(return_string[BK].replace('FETC?','')) #and remove the fetch commands from the string
   
    #print("Stripped return string: ",trunc_string) #print just to make sure [sometimes it is empty]
    #this try:...except is a way to catch errors without killing the program
  
        v1 = float(trunc_string[0]) #extract the resistance from the string & convert to float
        v2 = float(trunc_string[1])
    except:
        continue
        #This will raise an error (an exception) if the number is garbled or not present
   
    #Here we assume the conversion has gone OK:
    #print("resistance return string: [" ,return_bytes,"]")
    #The next line will plot points in blue
    #plt.scatter(v1,v2, color = 'b')
    #If you omit the color, you'll get something that looks like "Put in Me in the Zoo" by Robert Lopshire
    plt.scatter(v1,v2) #which is kind of handy for seeing the latest data point
    title("Raw data at {:4.1f} nm, count = {:d}".format(wavelength,count))
    plt.pause(0.005)
    print(count, strftime("%H:%M:%S",localtime()), "V1: {:9.3f}, V2 = {:9.4f}".format(v1,v2),end="\r")#no line feed
    v1_list.append(v1) #append it to the list
    v2_list.append(v2)
    #time_list.append(strftime("%H:%M:%S",localtime()))
    time_list.append(time.time())
    
    f_out.write("{:4d}\t{:12.4f}\t{:5e}\t{:5e}\n".format(count,time.time(), v1,v2))   #write out data in real time
    
    count+=1
    #savetxt(filename,strftime("%H:%M:%S",localtime()),end=" ")
    #savetxt(filename,tK(x,resistance),end=" ")
    #savet​-xt(filename,resistance,fmt="%9.4f\n\r")
    #while True:
        #plt.pause(0.05)
print("Scan complete",end ="\r")        
plt.show()

#print("Scan complete                                           ")    
#print("Closing B&K ports")
#for BK in range(NumBKports):
    #ser_BK[BK].close()
print("Closing file: "+f_out_root)  
f_out.close()


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Closing file: Blue22017.09.28.15.30.00.txt


### Print a list of v1 voltages

In [ ]:
v1_list  #just print a list of V1 voltages

### Make a simple plot

In [65]:
figure()
wavelength = 446
plot(v1_list, v2_list)
plot(v1_list,v2_list,'*g',ms=10)
title("Photocurrent vs. stopping potential at wavelength = {:4.1f} nm\n".format(wavelength)+f_out_root)
ylabel("Photocurrent")
xlabel("Stopping Voltage")
show()

### Just for fun, here's a simple model that suggests that the current increases exponentially below the stopping potential. We think this spread is caused by thermionic emission, in addition to the photoelectric effect

In [ ]:
from scipy.optimize import curve_fit
def func(x,m, k, offs, baselineslope): 
    return(m*exp(-k*x)-offs*ones(len(x))-baselineslope*x)

#popt is an array containing the optimized parameters.  Put in plausible starting guesses in p0:
p0 = (.1, 1, 0,0)
x = array(v1_list)
#print(x)
y = array(v2_list)
#print(y)
popt, pcov = curve_fit(func, x,y,p0) #this is the curve fit, return values are in popt
print(popt)
plt.figure()
plt.plot(x,y,'*b')
plt.plot(x,func(x,*p0),'--r')
plt.title("Model data and (bad) initial guess of parameters\n "+f_out_root)
finex = linspace(x[0],x[-1],1000)  #make a fine mesh upon which to plot the fit
plt.plot(finex,func(finex,popt[0],popt[1],popt[2], popt[3]), '-r') #plot the fit


###  But you'll probably want to determine the baseline of your photocurrent vs. stopping voltage data, which will allow you to estimate the point at which the most energetic electrons just begin to appear.  For that purpose, we illustrate how you might fit the flat baseline (changing the voltage for each color laser appropriately).  You can also see how flat the baseline is by taking the standard deviation.  Here we draw a second line that lies 1 standard deviation above the first.

In [68]:
#Illustration of how to find the indices of all v1 values > 1.0 (volts)
#Fit a straight line through this baseline
#Extract the slope and intercept of this line
from scipy.optimize import curve_fit
def line_fn(x,m,b):  #the straight line fitting function
    return m*x+b
x =array(v1_list)  #turn the list data into arrays
y =array(v2_list)
indx = x > 1.8  #look for values of v1 that are greater than 1.0 (this won't work with the bluest laser!)
x1 = x[indx]  #x1 is now the subset of v1 values that are greater than 1.0
y1 = y[indx]  #y1 is the corresponding set of v2 values
x1 = x1[6:]  #lopping off the last few points near high stopping potential
y1 = y1[6:]   #(Assumes you've scanned from high to low!)
figure()
plot(x1,y1,'*r')
p0 = (0,0)  #guess a slope and intercept.  If well warmed up and zeroed, this will be about right
popt, pcov = curve_fit(line_fn, x1,y1,p0) #this is the curve fit, return values are in popt
print(popt)  #print the actual intercept and slope

plt.plot(x,y,'*b')
plot(x1,y1,'*r')  #these red points were used in the baseline fit
plt.plot(x,line_fn(x,*popt),'--r') #and given a red dashed line

#just for reference, let's draw a green line that is one standard deviation above
#the baseline
onestd = std(y1)
print(onestd)
plt.plot(x, line_fn(x,popt[0],popt[1]+onestd),'-g')


#You might want to repeat this procedure for the vertical line as photocurrent rises rapidly.  Then find the intersection
#of the two lines.  That's one possible systematic way to determine the point at which the photocurrent starts to rise,
#but it probably underestimates the stopping voltage.

#Another possibility is to look for the first point at which the photocurrent rises above the noise level of this baseline


[-0.00079274 -0.00381507]
0.000158428937493


### Here's how you might open that last saved file, read in the data, and plot

In [20]:
#fname = f_out_root #here you'll need the full path:  "c:\\users\\yourname\\desktop\\filename"
#fname = "c:\\users\\tjossem\\desktop\\Charles Josh and Mark15.53.30.txt"
fname = "c:\\users\\yardasol\\desktop\\Lab 5\\Green2017.09.28.14.16.22.txt"
f_in = open(fname)
#f_in = open(f_out_root)
fcount = []
ftime = []
fv1 = []
fv2 = []
str_in =[]
for line in f_in.readlines():
    str_in = line.split() #chop it on white space
    #print(str_in)
    fcount.append(float(str_in[0]))
    ftime.append(float(str_in[1]))
    fv1.append(float(str_in[2]))
    fv2.append(float(str_in[3]))
f_in.close()
figure()
plot(fv1,fv2,'ob', ms = 12)
plot(fv1,fv2,'-k')
title("Photocurrent vs. stopping potential at wavelength = {:4.1f} nm".format(wavelength))
ylabel("Photocurrent")
xlabel("Stopping Voltage")
show()

### Saving a figure in pdf format is very simple--just give it a .pdf extension.  Other formats include .eps, .png, etc.  Don't worry that your plot files will look like xxxxx.txt.pdf

In [21]:
wavelength = 532.0
from scipy.optimize import curve_fit
def func(x,m, k, offs, baselineslope): 
    return(m*exp(-k*x)-offs*ones(len(x))-baselineslope*x)
from scipy.optimize import curve_fit
#popt is an array containing the optimized parameters.  Put in plausible starting guesses in p0:
p0 = (0.2, 2, 0,0)
x = array(fv1)
#print(x)
y = array(fv2)
#print(y)
popt, pcov = curve_fit(func, x,y,p0) #this is the curve fit, return values are in popt
print(popt)
plt.figure()
plt.plot(x,y,'*b')
plt.plot(x,func(x,*p0),'--r')
plt.title("Model data and (bad) initial guess of parameters\n ")
finex = linspace(x[0],x[-1],1000)  #make a fine mesh upon which to plot the fit
plt.plot(finex,func(finex,popt[0],popt[1],popt[2], popt[3]), '-g') #plot the fit

title("Photocurrent vs. stopping potential at wavelength = {:4.1f} nm\n Fit assumes a single exponential+baseline".format(wavelength))
ylabel("Photocurrent")
xlabel("Stopping Voltage")
savefig(fname+".pdf")

[ 2.13900969  3.88800694  0.10484743 -0.05277523]


In [63]:
#Put your function here
def V(x, a, b):
    return ((a*x) + b) #this is the R^2 function



In [109]:
W=(-1.7)*(1.6*10**(-19))
#L=array([638,638,638,532,532,532,446,446,446]) #wavelength data
#stop=array([0.69,0.68,0.64,1.03,1.01,0.94,1.52,1.62,1.57]) #Stopping voltage data
L=array([638, 532, 446])
stop=array([0.67, 0.993, 1.57])
independent=(3*10**(8))/((10**(-9))*L)


from scipy.optimize import curve_fit
#-4.14375*(10**(-15))
p0 = array([-4.14375*(10**(-15)), -1.7]) # my guesses at the parameters
popt, pcov = curve_fit(V, independent, stop)


plt.figure(3)
plt.scatter(independent, stop)
#plt.xlim(0, 2*(10**(-14)))
plt.plot(independent, V(independent, *popt), 'r')
xlabel("Frequency (s)")
ylabel("Stopping voltage (V)")
suptitle("Stopping voltage versus frequency")

print("Our fitting coefficients were a = ", format(popt[0]) + " +/- ", format(pcov[0,0]**0.5) + 
       ", b = {:6.3f}".format(popt[1]) + " +/- {:5.3f}".format(pcov[1,1]**0.5) )

Our fitting coefficients were a =  4.4689208267884675e-15 +/-  5.32674118069505e-16, b = -1.465 +/- 0.306
